In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import cPickle as pickle
#from model import utilities as ut

# Reading in static schedule data
stops_full = pd.read_csv('data/google_transit/stops.txt', index_col='stop_id')
routes = pd.read_csv('data/google_transit/routes.txt', index_col='route_id')
trips = pd.read_csv('data/google_transit/trips.txt', index_col='trip_id')
stop_times = pd.read_csv('data/google_transit/stop_times.txt')
shapes = pd.read_csv('data/google_transit/shapes.txt')

# Some of these stops are named "Not a public stop" but are still in trips.
# Luckily, in the few trips they appear in, they're only either at the
# beginning or the end, so we can remove them now and we'll still build
# a nice graph with the connections we expect.
stops = stops_full[~stops_full.index.isin([7520, 7530, 7531, 7540])]
stop_times = stop_times[~stop_times['stop_id'].isin([7520, 7530, 7531, 7540])]

# Oh and some stops are in stops.txt but not used in trips... let's remove 'em
used_stops = set(stop_times['stop_id'].unique())
stops = stops[stops.index.isin(used_stops)]

# Let's make some sorted stop-timepoint lists for each stop_id to
# make lookup faster for things
#all_stop_timepoints = {}
#for stopid in used_stops:
#    node_names = stop_times[stop_times['stop_id'] == stopid].\
#                    apply(lambda x: '{0}_{1}'.\
#                              format(stopid, x['arrival_time']),\
#                          axis=1)
#    all_stop_timepoints[stopid] = sorted(list(set(node_names)))

IOError: File data/google_transit/stops.txt does not exist

In [31]:
np.random.choice(stops.index)

3210

In [4]:
G_x = nx.read_gpickle('../../gimme-bus/graph_x_7.gpkl')

IOError: [Errno 2] No such file or directory: '../../gimme-bus/graph_x_7.gpkl'

In [38]:
sat_edges = [(u, v, d) for u,v,d in G_x.edges_iter(data=True) if d.get('service_id', 0) == 2]
sun_edges = [(u, v, d) for u,v,d in G_x.edges_iter(data=True) if d.get('service_id', 0) == 3]
w_edges = [(u, v, d) for u,v,d in G_x.edges_iter(data=True) if d.get('service_id', 0) == 1]

In [39]:
G_x.remove_edges_from(sat_edges)
G_x.remove_edges_from(sun_edges)

In [41]:
from model import utilities as ut

In [51]:
origin = 5874
o_lat = stops.loc[origin, 'stop_lat']
o_lon = stops.loc[origin, 'stop_lon']

start_time = '08:15:00'
start_node = ut.get_next_stop_timepoint(origin, start_time, all_stop_timepoints)

destination = 3210
d_lat = stops.loc[destination, 'stop_lat']
d_lon = stops.loc[destination, 'stop_lon']

est_time = np.ceil(1000.*ut.haversine(o_lon, o_lat, d_lon, d_lat)/3)

In [52]:
est_time

1504.0

In [78]:
path_time, path = ut.quickest_route(start_node, 3210, 2*est_time, G_x, all_stop_timepoints)

In [70]:
reload(ut)

<module 'model.utilities' from 'model/utilities.py'>

In [81]:
ut.nice_path(G_x, path, routes, condensed=True)

10:02:52: Arrive at Palou Ave & Keith St.
	  Take 44 to Keith St & Oakdale Ave
10:09:00: Arrive at Hudson Ave & 3rd St.
	  Walk to Third Street/Hudson/Innes
10:12:17: Arrive at Third Street/Hudson/Innes.
	  Take KT to Third Street & Evans Ave
10:20:00: Arrive at Third Street & 20th St.
	  Walk to 20th St & 3rd St
10:26:00: Arrive at 20th St & 3rd St.
	  Take 22 to Tennessee St & 18th St
10:32:08: Arrive at 17th St & De Haro St.
	  Walk to De Haro St & 17th St
10:33:03: Arrive at De Haro St & 17th St.
	  Take 19 to De Haro St & 16th St
10:35:33: Arrive at Division St & Townsend St.
	  Walk to 8th St & Townsend St
10:38:44: Arrive at 8th St & Townsend St
